In [1]:
import time

from osm_runner import Runner  # pip install osm-runner
import pandas as pd
import arcgis
from arcgis.features import FeatureLayer, GeoAccessor, GeoSeriesAccessor, SpatialDataFrame
from arcgis.geoenrichment import enrich
from arcgis import dissolve_boundaries
from arcgis.geometry import project
from arcgis.gis import GIS

from ipywidgets import *
from IPython.display import display

%config Completer.use_jedi = False 

In [2]:
print("ArcGIS UCSD Prompt...")
gis = GIS(username='mlabrado_dsc170fa20')

ArcGIS UCSD Prompt...
Enter password: ········


In [3]:
# Get it done dataset retrieved from san diego open data portal
# https://data.sandiego.gov/datasets/get-it-done-311/
gid_df = pd.read_csv("./data/get_it_done/get_it_done_2020_requests_datasd_v1.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
gid_sdf = pd.DataFrame.spatial.from_xy(gid_df, x_column="lng", y_column="lat")
gid_sdf

,service_request_id,service_request_parent_id,sap_notification_number,date_requested,case_age_days,service_name,case_record_type,date_closed,status,lat,...,council_district,comm_plan_code,comm_plan_name,park_name,case_origin,referred,public_description,iamfloc,floc,SHAPE
0,2837767,NaN,NaN,2020-01-01T00:00:00,1,Parking Zone Violation,Parking,2020-01-01T00:00:00,Closed,32.750250,...,2.0,23.0,Ocean Beach,NaN,Web,NaN,Large white converted schools bus. People livi...,SS-028127,SS-028127,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
1,2837768,NaN,NaN,2020-01-01T00:04:00,1,Parking Zone Violation,Parking,2020-01-01T00:00:00,Closed,32.753580,...,2.0,23.0,Ocean Beach,NaN,Web,NaN,"Tan Chevy RV, parked in dog beach parking lot ...",SS-028516,SS-028516,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
2,2837769,NaN,NaN,2020-01-01T00:06:00,1,Citizen Contact,Parking,2020-01-02T00:00:00,Closed,32.973030,...,5.0,3.0,Carmel Mountain Ranch,NaN,Mobile,NaN,Fireworks being set off next to dried out golf...,SS-028900,SS-028900,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
3,2837770,NaN,4.030023e+10,2020-01-01T00:12:00,16,Traffic Signal Light Out,TSW,2020-01-16T00:00:00,Closed,32.920253,...,6.0,15.0,Mira Mesa,NaN,Mobile,NaN,Yellow light out on traffic light facing west ...,SG-200735,SX-002477-008,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
4,2837771,2837769.0,NaN,2020-01-01T00:14:00,1,Citizen Contact,Parking,2020-01-02T00:00:00,Closed,32.975470,...,5.0,3.0,Carmel Mountain Ranch,NaN,Mobile,NaN,Fireworks set off land left at int. Windcrest/...,SS-028900,SS-028900,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274106,3175953,NaN,4.030037e+10,2020-12-16T22:24:00,0,Street Light Out,TSW,NaN,In Process,32.962751,...,5.0,29.0,Rancho Penasquitos,NaN,Web,NaN,Street light out at the intersection of Entrek...,PL-331408,SX-001090-001,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
274107,3175954,NaN,NaN,2020-12-16T22:28:00,0,Other,TSW,NaN,New,32.743075,...,3.0,42.0,Uptown,NaN,Mobile,NaN,"White male 5'10, 140bs slight beard wearing a ...",SS-004995,SS-004995,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
274108,3175956,NaN,NaN,2020-12-16T22:40:00,0,Engineering - Curb Evaluation,Traffic Engineering,NaN,New,32.837075,...,1.0,10.0,La Jolla,NaN,Mobile,NaN,NaN,SS-015522,SS-015522,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
274109,3175957,NaN,NaN,2020-12-16T22:48:00,0,72 Hour Violation,Parking,NaN,New,32.572821,...,8.0,25.0,Otay Mesa-Nestor,NaN,Mobile,NaN,Trailer left on street ongoing for months,SS-012775,SS-012775,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."


In [14]:
gid_sdf.columns

Index(['service_request_id', 'service_request_parent_id',
       'sap_notification_number', 'date_requested', 'case_age_days',
       'service_name', 'case_record_type', 'date_closed', 'status', 'lat',
       'lng', 'street_address', 'zipcode', 'council_district',
       'comm_plan_code', 'comm_plan_name', 'park_name', 'case_origin',
       'referred', 'public_description', 'iamfloc', 'floc', 'SHAPE'],
      dtype='object')

In [15]:
gid_sdf_clean = gid_sdf[["date_requested", "service_name", "case_record_type", "status", "lat", "lng", "comm_plan_name", "public_description", "SHAPE"]]

In [36]:
gid_sdf_clean.spatial.to_featurelayer("full_gid_test1")

<Item title:"full_gid_test1" type:Feature Layer Collection owner:mlabrado_dsc170fa20>

In [52]:
gid_sdf_clean.loc[:, "service_name"].value_counts()

Illegal Dumping                                         36868
Missed Collection                                       32897
Encampment                                              26196
Graffiti Removal                                        26004
72 Hour Violation                                       24501
                                                        ...  
PAVEMENT RESTORE CONCRETE UTILITY TRENCH                    1
ROW MAINTENANCE                                             1
TREE EVALUATE TREE FOR REMOVAL                              1
Quality of Life Issues Infestation                          1
ENGINEERING - TRAFFIC SAFETY REQUEST FOR INFORMATION        1
Name: service_name, Length: 224, dtype: int64

In [53]:
gid_sdf_clean[["public_description"]].value_counts()

public_description                                                                                                                                           
Scooters blocking sidewalk! Tripping hazard! Not coral!                                                                                                          2670
Graffiti                                                                                                                                                         1473
Couch                                                                                                                                                             799
Mattress                                                                                                                                                          795
No Recycling                                                                                                                                                      598
            

# Sample 50,000 service requests at random to get more managable dataset

In [42]:
import numpy as np

In [46]:
random_indices = np.random.choice(gid_sdf.shape[0], 50000)

In [49]:
random_50k_gid_sdf = gid_sdf.iloc[random_indices][["date_requested", "service_name", "case_record_type", "comm_plan_name","public_description", "SHAPE"]]
random_50k_gid_sdf

,date_requested,service_name,case_record_type,comm_plan_name,public_description,SHAPE
226321,2020-10-20T09:20:00,Traffic Sign - Maintain,TSW,Peninsula,NaN,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
10909,2020-01-13T15:00:00,72 Hour Violation,Parking,Skyline-Paradise Hills,Broke down (?) & in the same spot for 3 weeks ...,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
52817,2020-03-02T08:43:00,Shared Mobility Device,ESD Complaint/Report,Mission Beach,Corral is across the street,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
475,2020-01-01T19:05:00,Shared Mobility Device,ESD Complaint/Report,Downtown,Scooters blocking sidewalk! Tripping hazard! N...,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
194335,2020-09-17T09:43:00,Street Sweeping,TSW,Pacific Beach,By CVS in Pacific Beach the curbside is awash ...,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
...,...,...,...,...,...,...
257662,2020-11-27T09:06:00,Pothole,TSW,Otay Mesa,Big potholes up and down street,"{""spatialReference"": {""wkid"": 4326}, ""x"": -116..."
255110,2020-11-23T14:18:00,72 Hour Violation,Parking,Pacific Beach,Motorhome recurring long term parking at inter...,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
92438,2020-05-04T19:21:00,Drain Maintenance,TSW,Clairemont Mesa,Sewage/sulphurous smell that's extremely ranci...,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."
184210,2020-09-04T10:22:00,Illegal Dumping,ESD Complaint/Report,Mid-City:City Heights,Folding table,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117..."


In [51]:
random_50k_gid_sdf.spatial.to_featurelayer('random_50k_gid_sdf')

<Item title:"random_50k_gid_sdf" type:Feature Layer Collection owner:mlabrado_dsc170fa20>